**动量优化**：使用在动量步骤之前计算的梯度
1. $m\leftarrow \beta m-\eta\Delta_{\theta}J(\theta)$
2. $\theta\leftarrow \theta +m$

```python
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9)
```

**Nesterov动量优化**：使用在动量步骤之后计算的梯度，收敛更快
1. $m\leftarrow \beta m-\eta\Delta_{\theta}J(\theta+\beta m)$
2. $\theta\leftarrow \theta +m$

```python
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)
```

**AdaGrad**：该算法会降低学习率，但是对于陡峭的维度，它的执行速度要比对缓慢下降的维度的执行速度要快。这称为自适应学习率。它有助于将结果更新更直接地指向全局最优解。另一个好处是，它几乎不需要调整学习率超参数$\eta$。
1. $ s \leftarrow s + \nabla_\theta J(\theta) \otimes \nabla_\theta J(\theta) $

2. $ \theta \leftarrow \theta - \eta \, \nabla_\theta J(\theta) \otimes \frac{1}{\sqrt{s + \varepsilon}} $

对于简单的二次问题，AdaGrad经常表现良好，但是在训练神经网
络时，它往往停止得太早。

**RMSProp**：
1. $ s \leftarrow \beta s + (1-\beta)\nabla_\theta J(\theta) \otimes \nabla_\theta J(\theta) $
2. $ \theta \leftarrow \theta - \eta \, \nabla_\theta J(\theta) \otimes \frac{1}{\sqrt{s + \varepsilon}} $

衰减率β通常设置为0.9。
```python
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9)
```

**Adam优化**：它跟踪过去梯度的指数衰减平均值和过去平方梯度的指数衰减平均值
1. $ \mathbf{m} \leftarrow \beta_1 \mathbf{m} - (1 - \beta_1) \, \nabla_\theta J(\theta) $

2. $ \mathbf{s} \leftarrow \beta_2 \mathbf{s} + (1 - \beta_2) \, \nabla_\theta J(\theta) \otimes \nabla_\theta J(\theta) $

3. $ \hat{\mathbf{m}} \leftarrow \dfrac{\mathbf{m}}{1 - \beta_1^t} $

4. $ \hat{\mathbf{s}} \leftarrow \dfrac{\mathbf{s}}{1 - \beta_2^t} $

5. $ \theta \leftarrow \theta - \eta \, \hat{\mathbf{m}} \otimes \dfrac{1}{\sqrt{\hat{\mathbf{s}} + \varepsilon}} $

> 在此等式中，$ t $ 表示迭代次数（从 1 开始）。

```python
optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
```


### 学习率调度

**幂调度**：$\eta(t)=\dfrac{\eta_{0}}{(1+\frac{t}{s})^c}$

```python
optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-4)
```

**指数调度**：$\eta(t)=\eta_{0}(0.1)^{\frac{t}{s}}$

```python
def exponential_decay_fn(epoch):
    return 0.01 * 0.1 ** (epoch / 20)
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 ** (epoch / s)
    return exponential_decay_fn
exponential_decay_fn = exponential_decay(lr0=0.01, s=20)
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)
history = model.fit(X_train_scaled, y_train, [...], callbacks=[lr_scheduler])
```

**分段恒定调度**：对一些轮次使用恒定的学习率，对于另外一些轮次使用较小的学习率

```python
def piecewise_constant_fn(epoch):
    if epoch < 5:
        return 0.01
    elif epoch < 15:
        return 0.005
    else:
        return 0.001
```

**性能调度**：每$N$步测量一次验证误差（就像提前停止一样），并且当误差停止
下降时，将学习率降低$\lambda$倍。

```python
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
```

**1周期调度**：从提高初始学习率$\eta_0$开始，在训练中途线性增长至$\eta_1$。然
后，它在训练的后半部分将学习率再次线性降低到$\eta_0$，通过将学习率降低几个数量级来完成最后几个轮次。